In [1]:
#net = torch.load("pruned.pkl")
#loading libraries
import torch as torch
import torch.nn as nn
import pandas as pd
import numpy as np
from torch.autograd import Variable
from sklearn.metrics import accuracy_score
import torch.nn.functional as F

In [2]:
class MaskedLinear(nn.Linear):
    def __init__(self, in_features, out_features, bias=True):
        super(MaskedLinear, self).__init__(in_features, out_features, bias)
        self.mask_flag = False
    
    def set_mask(self, mask):
        self.register_buffer('mask', mask)
        mask_var = self.get_mask()
        self.weight.data = self.weight.data*mask_var.data
        self.mask_flag = True
    
    def get_mask(self):
        # print(self.mask_flag)
        return Variable(self.mask, requires_grad=False)
    
    def forward(self, x):
        if self.mask_flag == True:
            mask_var = self.get_mask()
            weight = self.weight * mask_var
            return F.linear(x, weight, self.bias)
        else:
            return F.linear(x, self.weight, self.bias)

In [3]:
#network definition
class Net(nn.Module):
    def __init__(self,input_size,hidden_size,num_classes):
        super(Net, self).__init__()
        self.fc1 = MaskedLinear(input_size, hidden_size,bias=1) 
        self.tanh = nn.Tanh()
        self.fc2 = MaskedLinear(hidden_size, num_classes,bias=1)
        self.sigm=nn.Sigmoid()
    
    
    def forward(self,x):
        x=self.fc1(x)
        x=self.tanh(x)
        out=self.fc2(x)
        out=self.sigm(out)
        
        return out
    def store_neuron_outputs(self,x):
        x=self.fc1(x)
        x=self.tanh(x)
        
        return x
        
    def set_masks(self, masks):
        self.fc1.set_mask(masks[0])
        self.fc2.set_mask(masks[1])

In [8]:
datatrain=pd.read_csv('Iris.csv')
datatrain.loc[datatrain['Species']=='Iris-setosa', 'Species']=0
datatrain.loc[datatrain['Species']=='Iris-versicolor', 'Species']=1
datatrain.loc[datatrain['Species']=='Iris-virginica', 'Species']=2
datatrain = datatrain.apply(pd.to_numeric)
datatrain=np.array(datatrain)
datatrain=datatrain[:,1:6]


#split x and y (feature and target)
X = datatrain[:,0:4]
y = datatrain[:,4]

#y=pd.get_dummies(y)
#y=np.array(y)
net = torch.load("pruned.pkl")

In [16]:
net

Net(
  (fc1): MaskedLinear(in_features=4, out_features=1, bias=True)
  (tanh): Tanh()
  (fc2): MaskedLinear(in_features=1, out_features=3, bias=True)
  (sigm): Sigmoid()
)

In [9]:
X_data= Variable(torch.Tensor(X)).float()
out=net(X_data).data.numpy()
_, predicted = torch.max(net(X_data).data, 1)
for index,row in enumerate(datatrain):
    row[-1]=predicted.numpy()[index]
dataset=(datatrain,out)
datatrain=datatrain[:,0:4]

In [14]:
from sklearn.metrics import accuracy_score

In [15]:
accuracy_score(predicted.numpy(),y)

1.0

In [171]:
def SSE_value(teaching_outputs,NN_outputs):
    # teaching_outputs and NN_outputs is of size mxn where m=no. of examples and n is no. of classes in output vector.
    sse=np.sum(np.square(teaching_outputs-NN_outputs))
    return sse



In [172]:
no_of_classes=3
def get_majority_class(data):
    X_data_actual=Variable(torch.Tensor(data)).float()
    actual_out=net(X_data_actual).data.numpy()
    _, actual_predicted = torch.max(net(X_data_actual).data, 1)
    outcomes=list(actual_predicted.numpy())
    return max(set(outcomes), key=outcomes.count)

def get_NNoutputs(data):
    X_data_actual=Variable(torch.Tensor(data)).float()
    actual_out=net(X_data_actual).data.numpy()
    _, actual_predicted = torch.max(net(X_data_actual).data, 1)
    return actual_out

# Calculate the SSE index for a split dataset
def SSE_index(groups):
    # count all samples at split point
    no_of_instances=len(groups[0])+len(groups[1])
    
    sse_value_sum = 0.0
    
    for group in groups:
        size=len(group)
        sse_value=0.0
        
        # avoid divide by zero
        if size == 0:
            continue
            
        maj_class=get_majority_class(group)
        
        one_hot_teaching=np.zeros((group.shape[0],no_of_classes))
        for i in range(len(one_hot_teaching)):
            one_hot_teaching[i][maj_class]=1
        N_N_outputs=get_NNoutputs(group)
        
        sse_value=size*SSE_value(one_hot_teaching,N_N_outputs)/no_of_instances
        
        sse_value_sum=sse_value_sum+sse_value
        
    return sse_value_sum

In [173]:
# Split a dataset based on an attribute and an attribute value
def test_split(index, value, data):
    left, right = list(), list()
    for row in data:
        if row[index] < value:
            left.append(row)
        else:
            right.append(row)
    return np.array(left), np.array(right)

In [174]:

def best_feature(data):
    sub_data=data.copy()
    feature_index=0
    max_cost=0
    for i in range(0,np.size(sub_data,1)):
        zero_feature_data=sub_data.copy()
        zero_feature_data[:,i]=0
        
        X_data_zero= Variable(torch.Tensor(zero_feature_data)).float()
        zero_out=net(X_data).data.numpy()
        _, zero_predicted = torch.max(net(X_data_zero).data, 1)
        
        X_data_actual=Variable(torch.Tensor(sub_data)).float()
        actual_out=net(X_data_actual).data.numpy()
        _, actual_predicted = torch.max(net(X_data_actual).data, 1)
        
        one_hot_teaching=np.zeros_like(zero_out)
        for j in range(len(zero_predicted)):
            one_hot_teaching[j][zero_predicted[j]]=1
            
        cost=SSE_value(one_hot_teaching,zero_out)
        print(cost)
        if max_cost<cost:
            max_cost=cost
            feature_index=i
    #print(feature_index)   
    return feature_index

In [175]:

# Select the best split point for a dataset for a feature
def get_split(data,feature):
    b_value, b_score, b_groups = 999, 999, None
    for row in data:
        groups = test_split(feature, row[feature], data)
        sse = SSE_index(groups)
        if sse < b_score:
            b_value, b_score, b_groups = row[feature], sse, groups
    return {'index':feature, 'value':b_value, 'groups':b_groups}

In [176]:
def to_terminal(group):
    X_data_actual=Variable(torch.Tensor(group)).float()
    actual_out=net(X_data_actual).data.numpy()
    _, actual_predicted = torch.max(net(X_data_actual).data, 1)
    outcomes=list(actual_predicted.numpy())
    return max(set(outcomes), key=outcomes.count)

In [177]:
def split(node, max_depth, min_size, depth):
    left, right = node['groups']
    del(node['groups'])
    # check for a no split
    if not left.tolist() or not right.tolist():
        print(left.shape)
        print(right.shape)
        if(left.shape[0]==0 and right.shape[0]!=0):
            node['left'] = node['right'] = to_terminal(right)
        elif(right.shape[0]==0 and left.shape[0]!=0):
            node['left'] = node['right'] = to_terminal(left)
        else:
            node['left'] = node['right'] = to_terminal(np.concatenate((left,right),axis=0))
        return
    # check for max depth
    if depth >= max_depth:
        node['left'], node['right'] = to_terminal(left), to_terminal(right)
        return
    # process left child
    if len(left) <= min_size:
        node['left'] = to_terminal(left)
    else:
        b_feature=best_feature(left)
        node['left'] = get_split(left,b_feature)
        split(node['left'], max_depth, min_size, depth+1)
    # process right child
    if len(right) <= min_size:
        node['right'] = to_terminal(right)
    else:
        b_feature=best_feature(right)
        node['right'] = get_split(right,b_feature)
        split(node['right'], max_depth, min_size, depth+1)

In [178]:

# Build a decision tree
def build_tree(train, max_depth, min_size):
    feature=best_feature(train)
    root = get_split(train,feature)
    split(root, max_depth, min_size, 1)
    return root

In [179]:
# Make a prediction with a decision tree
def predict(node, row):
    if row[node['index']] < node['value']:
        if isinstance(node['left'], dict):
            return predict(node['left'], row)
        else:
            return node['left']
    else:
        if isinstance(node['right'], dict):
            return predict(node['right'], row)
        else:
            return node['right']

In [180]:
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    
    print(correct / float(len(actual)) * 100.0)

In [181]:
tree = build_tree(datatrain, 10, 5)

34.25941
34.25941
166.46089
100.03233
73.24859
73.24859
91.53776
81.30214
66.552155
66.552155
66.552155
66.552155
88.507774
88.507774
88.507774
88.507774
100.47806
100.47806
100.47806
100.47806
107.46632
107.46632
107.46632
107.46632
109.4629
109.4629
109.4629
109.4629
104.47179
104.47179
104.47179
104.47179
108.464714
108.464714
108.464714
108.464714
(0,)
(8, 4)
94.49185
94.49185
94.49185
94.49185
105.469696
105.469696
105.469696
105.469696
108.464714
108.464714
108.464714
108.464714
(0,)
(8, 4)
105.469696
105.469696
105.469696
105.469696
110.461044
110.461044
110.461044
110.461044
123.14581
123.14581
91.500854
109.22098
119.931526
119.931526
103.473175
108.53776
118.59224
118.59224
108.464714
113.52725
119.66367
119.66367
104.47179
117.13343
118.32439
118.32439
109.4629
115.791695
172.12335
172.12335
91.53776
137.27081
139.46724
139.46724
82.519264
125.55653
127.592255
127.592255
99.479965
121.00296
119.788666
119.788666
109.4629
118.32439
124.25296
124.25296
106.46788
119.12796
128.

In [182]:
tree

{'index': 2,
 'value': 4.4,
 'left': {'index': 2,
  'value': 3.0,
  'left': {'index': 0,
   'value': 5.1,
   'left': {'index': 0,
    'value': 4.9,
    'left': {'index': 0,
     'value': 4.7,
     'left': {'index': 0, 'value': 4.6, 'left': 0, 'right': 0},
     'right': {'index': 0, 'value': 4.8, 'left': 0, 'right': 0}},
    'right': {'index': 0,
     'value': 5.0,
     'left': 0,
     'right': {'index': 0, 'value': 5.0, 'left': 0, 'right': 0}}},
   'right': {'index': 0,
    'value': 5.3,
    'left': {'index': 0,
     'value': 5.2,
     'left': {'index': 0, 'value': 5.1, 'left': 0, 'right': 0},
     'right': 0},
    'right': {'index': 0,
     'value': 5.5,
     'left': {'index': 0, 'value': 5.4, 'left': 0, 'right': 0},
     'right': 0}}},
  'right': {'index': 0,
   'value': 5.7,
   'left': {'index': 0,
    'value': 5.5,
    'left': 1,
    'right': {'index': 0, 'value': 5.6, 'left': 1, 'right': 1}},
   'right': {'index': 0,
    'value': 5.9,
    'left': {'index': 0, 'value': 5.8, 'left':

In [183]:
predicted_1=[]
for row in datatrain:
    prediction = predict(tree, row)
    predicted_1.append(prediction)
    print('Got=%d' % (prediction))

Got=0
Got=0
Got=0
Got=0
Got=0
Got=0
Got=0
Got=0
Got=0
Got=0
Got=0
Got=0
Got=0
Got=0
Got=0
Got=0
Got=0
Got=0
Got=0
Got=0
Got=0
Got=0
Got=0
Got=0
Got=0
Got=0
Got=0
Got=0
Got=0
Got=0
Got=0
Got=0
Got=0
Got=0
Got=0
Got=0
Got=0
Got=0
Got=0
Got=0
Got=0
Got=0
Got=0
Got=0
Got=0
Got=0
Got=0
Got=0
Got=0
Got=0
Got=2
Got=2
Got=2
Got=1
Got=2
Got=1
Got=2
Got=1
Got=1
Got=1
Got=1
Got=1
Got=1
Got=1
Got=1
Got=2
Got=1
Got=1
Got=2
Got=1
Got=2
Got=1
Got=2
Got=1
Got=1
Got=1
Got=2
Got=2
Got=1
Got=1
Got=1
Got=1
Got=1
Got=1
Got=1
Got=1
Got=2
Got=2
Got=1
Got=1
Got=1
Got=1
Got=1
Got=1
Got=1
Got=1
Got=1
Got=1
Got=1
Got=1
Got=2
Got=2
Got=2
Got=2
Got=2
Got=2
Got=1
Got=2
Got=2
Got=2
Got=2
Got=2
Got=2
Got=1
Got=2
Got=2
Got=2
Got=2
Got=2
Got=1
Got=2
Got=1
Got=2
Got=2
Got=2
Got=2
Got=2
Got=1
Got=2
Got=2
Got=2
Got=2
Got=2
Got=2
Got=1
Got=2
Got=2
Got=2
Got=1
Got=2
Got=2
Got=2
Got=2
Got=2
Got=2
Got=2
Got=2
Got=2
Got=2
Got=2


In [186]:
accuracy_metric(predicted.numpy(),predicted_1)

88.66666666666667